In [16]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [28]:
# This section is just me figuring out how tensors work
lower_bound = tf.zeros(shape=[1,1]) # Lower bound u(0)=0
upper_bound = tf.ones(shape=[1,1]) # Upper bound u(1)=1
x = tf.random.uniform(shape=[1,100]) # Interior points
training_pts = tf.concat([lower_bound, x, upper_bound], 1) # Actual training data
x_0 = tf.slice(training_pts, [0,0], [1,1])
x_1 = tf.slice(training_pts, [0,101], [1,1])
print(x_0.numpy(), x_1.numpy())

[[0.]] [[1.]]


In [40]:
# Construct Simple Feedforward Network
model = Sequential()
model.add(Dense(100, batch_size=100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1))
#model.compile(loss='mean_squared_error', optimizer='adam')

In [41]:
# Define Loss Function
def loss_fn(x):
    with tf.GradientTape() as t:
        t.watch(x)
        f = model(x)
    df_dx = t.gradient(f,x)
    u_0 = tf.zeros(shape=[1,1]) # Lower boundary condition
    u_1 = tf.ones(shape=[1,1]) # Upper boundary condition
    bound_weight = 100 
    mse = tf.keras.losses.MeanSquaredError()
    lower_bound_error = tf.math.subtract(tf.slice(f,[0,0],[1,1]),u_0)
    upper_bound_error = tf.math.subtract(tf.slice(f,[0,99],[1,1]),u_1)
    loss = mse(df_dx, bound_weight*lower_bound_error, bound_weight*upper_bound_error)
    return loss

In [35]:
# Train network
optimizer = tf.keras.optimizers.Adam() # Fancy gradient decent
epochs = 5 
iterations_per_epoch = 5
minibatch_size = 100 # Number of points to be selected each iteration
for epoch in range(epochs):
    for iteration in range(iterations_per_epoch):
        lower_bound = tf.zeros(shape=[1,1]) # Lower bound u(0)=0
        upper_bound = tf.ones(shape=[1,1]) # Upper bound u(1)=1
        interior_pts = tf.random.uniform(shape=[1,minibatch_size-2]) # Interior points
        x = tf.concat([lower_bound, interior_pts, upper_bound], 1) # Actual training data
        with tf.GradientTape() as t:
            t.watch(x)
            f = model(x, training=True) # Estimate for u
            loss = loss_fn(x) # Loss
        grads = t.gradient(loss, model.trainable_weights) # Find model gradients
        optimizer.apply_gradients(zip(grads,model.trainable_weights)) # Perform gradient decent

ValueError: Input 0 of layer sequential_4 is incompatible with the layer: expected axis -1 of input shape to have value 1 but received input with shape (1, 100)